In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
WITH usa_tracks AS
        (SELECT  
            invoice_line.*
         FROM invoice_line
         INNER JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
         INNER JOIN customer ON customer.customer_id = invoice.customer_id
         WHERE customer.country = 'USA')
    
SELECT 
    genre.name AS genre,
    COUNT(usa_tracks.invoice_line_id) AS usa_count,
    CAST(COUNT(usa_tracks.invoice_line_id) AS FLOAT)/
        (SELECT COUNT(*) FROM usa_tracks
        ) AS percentage
FROM
    usa_tracks
INNER JOIN track ON usa_tracks.track_id = track.track_id
INNER JOIN genre ON genre.genre_id = track.genre_id
GROUP BY genre
ORDER BY usa_count DESC;


Done.


genre,usa_count,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


The top three genres sold in the USA are Rock, Alternative & Punk, and Metal. The only artist in those top three genres is Red Tone. The other artists that are high on the list of genres are Slim Jim Bites and then Meteor and the Girls.

In [4]:
%%sql

SELECT
    employee.first_name || " " || employee.last_name AS name,
    employee.hire_date,
    SUM(invoice.total) AS total_sales
FROM employee
INNER JOIN customer ON employee.employee_id = customer.support_rep_id
INNER JOIN invoice ON customer.customer_id = invoice.customer_id
GROUP BY employee.employee_id



Done.


name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


All of the employees are close in sales to eachother. Steve has the lowest total sales but he has been at the company for less time. Jane has the most sales and she has been with the company the longest.

In [11]:
%%sql
WITH  country_info AS  
    (SELECT   
        CASE
            WHEN 
                (SELECT count(*)
                 FROM customer
                 WHERE country = cust.country) = 1
            THEN "Other"
            ELSE cust.country
            END AS country,
            customer.customer_id,
            invoice_line.*
        FROM invoice_line
        INNER JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
        INNER JOIN customer AS cust ON cust.customer_id = invoice.customer_id)

SELECT
    country,
    total_customers,
    total_sales,
    average_sale,
    average_value
FROM    
(SELECT
    country,
    COUNT(DISTINCT customer_id) AS total_customers,
    SUM(unit_price) AS total_sales,
    SUM(unit_price)/COUNT(DISTINCT customer_id) AS average_sale,
    SUM(unit_price)/COUNT(DISTINCT invoice_id) AS average_value,
    CASE
        WHEN
            country = 'Other' THEN 1
        ELSE 0
        END AS sort
FROM
    country_info
GROUP BY country
ORDER BY sort, total_sales desc)
    

(sqlite3.OperationalError) no such column: customer.customer_id
[SQL: WITH  country_info AS  
    (SELECT   
        CASE
            WHEN 
                (SELECT count(*)
                 FROM customer
                 WHERE country = cust.country) = 1
            THEN "Other"
            ELSE cust.country
            END AS country,
            customer.customer_id,
            invoice_line.*
        FROM invoice_line
        INNER JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
        INNER JOIN customer AS cust ON cust.customer_id = invoice.customer_id)

SELECT
    country,
    total_customers,
    total_sales,
    average_sale,
    average_value
FROM    
(SELECT
    country,
    COUNT(DISTINCT customer_id) AS total_customers,
    SUM(unit_price) AS total_sales,
    SUM(unit_price)/COUNT(DISTINCT customer_id) AS average_sale,
    SUM(unit_price)/COUNT(DISTINCT invoice_id) AS average_value,
    CASE
        WHEN
            country = 'Other' THEN 1
        ELSE 0
    